In [1]:
import pandas as pd

# Assuming df1 and df2 are your DataFrames
df1=pd.read_csv("precipitation_calculated_zones.csv")
df2=pd.read_csv("Merra2(tem,hum,wind)_mean_calculated_zones.csv")

In [2]:
df1.head()

,system:index,precipitationCal,system:time_start,zone_name,.geo
0,20170101000000_00000000000000000002,0.0,2017-01-01,Greater Hyderabad Municipal Corporation North ...,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,20170101000000_00000000000000000004,0.0,2017-01-01,Greater Hyderabad Municipal Corporation Centra...,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,20170101000000_00000000000000000000,0.0,2017-01-01,Greater Hyderabad Municipal Corporation East Zone,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,20170101000000_00000000000000000003,0.0,2017-01-01,Greater Hyderabad Municipal Corporation West Zone,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,20170101000000_00000000000000000001,0.0,2017-01-01,Greater Hyderabad Municipal Corporation South ...,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [3]:
df2.head()

,system:index,QV10M_mean,T2M_mean,U10M_mean,V10M_mean,system:time_start,zone_name,.geo
0,2017010100_00000000000000000002,0.010099,287.111211,-0.619581,2.728395,2017-01-01,Greater Hyderabad Municipal Corporation North ...,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,2017010100_00000000000000000004,0.010700,287.246674,-0.570899,2.230994,2017-01-01,Greater Hyderabad Municipal Corporation Centra...,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,2017010100_00000000000000000000,0.010528,287.207951,-0.584815,2.373179,2017-01-01,Greater Hyderabad Municipal Corporation East Zone,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,2017010100_00000000000000000003,0.010162,287.125410,-0.614478,2.676258,2017-01-01,Greater Hyderabad Municipal Corporation West Zone,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,2017010100_00000000000000000001,0.010700,287.246674,-0.570899,2.230994,2017-01-01,Greater Hyderabad Municipal Corporation South ...,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [4]:
# Convert the 'system:time_start' column to datetime format
df1['system:time_start'] = pd.to_datetime(df1['system:time_start'])
df2['system:time_start'] = pd.to_datetime(df2['system:time_start'])

# Merge the two DataFrames on 'system:time_start'
# Merge the dataframes on 'zone_name' and 'system:time_start'
merged_df = pd.merge(df1, df2, on=['zone_name', 'system:time_start'])


In [5]:

# Select only the desired columns
# Select the desired columns including 'zone_name'
merged_df = merged_df[['system:time_start', 'zone_name', 'QV10M_mean', 'T2M_mean', 'U10M_mean', 'V10M_mean', 'precipitationCal']]

# Rename columns
merged_df.rename(columns={
    'system:time_start': 'Date',
    'zone_name': 'Zone_Name',
    'QV10M_mean': 'Specific_Humidity_10m',
    'T2M_mean': 'Air_Temperature_2m',
    'U10M_mean': 'Eastward_Wind_10m',
    'V10M_mean': 'Northward_Wind_10m',
    'precipitationCal': 'Precipitation_mm'
}, inplace=True)



In [6]:
merged_df.head()

,Date,Zone_Name,Specific_Humidity_10m,Air_Temperature_2m,Eastward_Wind_10m,Northward_Wind_10m,Precipitation_mm
0,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.010099,287.111211,-0.619581,2.728395,0.0
1,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.010349,287.393650,-0.551143,2.600508,0.0
2,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.010517,291.022516,-0.609978,3.245707,0.0
3,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.010131,294.511585,-0.921855,2.940640,0.0
4,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.008208,298.493965,-1.441991,2.395005,0.0


In [7]:
# Convert temperature from Kelvin to Celsius
merged_df['Air_Temperature_2m'] -= 273.15  # Convert Kelvin to Celsius

# Convert wind speed from meters per second to kilometers per hour
merged_df['Eastward_Wind_10m'] *= 3.6  # Convert m/s to km/h
merged_df['Northward_Wind_10m'] *= 3.6

In [8]:
merged_df.head()

,Date,Zone_Name,Specific_Humidity_10m,Air_Temperature_2m,Eastward_Wind_10m,Northward_Wind_10m,Precipitation_mm
0,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.010099,13.961211,-2.230491,9.822222,0.0
1,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.010349,14.243650,-1.984114,9.361827,0.0
2,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.010517,17.872516,-2.195920,11.684546,0.0
3,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.010131,21.361585,-3.318680,10.586305,0.0
4,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.008208,25.343965,-5.191167,8.622018,0.0


In [9]:
# Group by 'Date' and 'Zone_Name', and aggregate using sum for 'Precipitation_mm' and mean for other columns
aggregated_data = merged_df.groupby(['Date', 'Zone_Name']).agg({'Specific_Humidity_10m': 'mean',
                                                           'Air_Temperature_2m': 'mean',
                                                           'Eastward_Wind_10m': 'mean',
                                                           'Northward_Wind_10m': 'mean',
                                                           'Precipitation_mm': 'max'}).reset_index()


In [10]:

# Print the resulting DataFrame
aggregated_data.head(20)

,Date,Zone_Name,Specific_Humidity_10m,Air_Temperature_2m,Eastward_Wind_10m,Northward_Wind_10m,Precipitation_mm
0,2017-01-01,Greater Hyderabad Municipal Corporation Centra...,0.007663,20.864690,-4.095102,6.032706,0.0
1,2017-01-01,Greater Hyderabad Municipal Corporation East Zone,0.007646,20.864623,-4.035985,6.143598,0.0
2,2017-01-01,Greater Hyderabad Municipal Corporation North ...,0.007604,20.864455,-3.888297,6.420636,0.0
3,2017-01-01,Greater Hyderabad Municipal Corporation South ...,0.007663,20.864690,-4.095102,6.032706,0.0
4,2017-01-01,Greater Hyderabad Municipal Corporation West Zone,0.007610,20.864480,-3.909974,6.379974,0.0
5,2017-01-02,Greater Hyderabad Municipal Corporation Centra...,0.007799,21.624963,-3.187930,0.430364,0.0
6,2017-01-02,Greater Hyderabad Municipal Corporation East Zone,0.007729,21.631686,-2.936640,0.292032,0.0
7,2017-01-02,Greater Hyderabad Municipal Corporation North ...,0.007553,21.648481,-2.308854,-0.053558,0.0
8,2017-01-02,Greater Hyderabad Municipal Corporation South ...,0.007799,21.624963,-3.187930,0.430364,0.0
9,2017-01-02,Greater Hyderabad Municipal Corporation West Zone,0.007579,21.646016,-2.400997,-0.002834,0.0


In [12]:
# Save merged_df as precipitation data
aggregated_data.to_csv('finalprecipitation_data.csv', index=False)
